# Анализатор текстовых пресс-релизов

Задача: На основании исторических пресс-релизов кредитных рейтинговых агентств участникам хакатона необходимо построить интерпретируемую ML-модель, устанавливающую взаимосвязь между текстом пресс-релиза и присвоенным кредитным рейтингом по национальной рейтинговой шкале Российской Федерации для организации с учетом методологических особенностей оценки рейтинга. ML-модель должна не просто устанавливать соответствие текста пресс-релиза кредитному рейтингу, но также и выделять ключевые конструкции в тексте, соответствующие присвоенному кредитному рейтингу.



## План

Исходя из поставленной зажачи мы сформировали план
1. Загрузить данные
2. Оценить данные, если нужно, почистить, лематизировать
3. Использовать для токенизации инструмент TfidfVectorizer
4. Построить на основе полученных векторов разные модели предсказания кредитного рейтинга
5. Использовать трансформер BERT
6. Построить на основе полученных векторов разные модели предсказания кредитного рейтинга
7. Оценить результаты и сделать выводы

In [15]:
import numpy as np
import pandas as pd
import re
import torch
from tqdm import notebook
from sklearn.model_selection import train_test_split, GridSearchCV
from pymystem3 import Mystem
m = Mystem()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import torch
import transformers as ppb
import lightgbm as lgb
from pytorch_transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import wordnet
import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import word_tokenize
import requests
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.probability import FreqDist
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

from lightgbm import LGBMClassifier

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [96]:
data = pd.read_excel('CRA_train_1200.xlsx')

In [99]:
data.head(5)


,Id,text,category,rating
0,1,Повышение кредитного рейтинга Акционерного об...,A,A
1,2,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
2,3,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
3,4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA
4,5,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB


In [98]:
data = data.rename(columns={
    'pr_txt': 'text',
    'Категория': 'category',
    'Уровень рейтинга': 'rating'
})


In [103]:
import requests
import json

# Замените на свой OAuth-токен
OAUTH_TOKEN = "y0_AgAAAAAFvktuAATuwQAAAADsR6xABGN80xXHTc682VqotLAtG1s7dvY"

response = requests.post(
    "https://iam.api.cloud.yandex.net/iam/v1/tokens",
    data=json.dumps({"yandexPassportOauthToken": OAUTH_TOKEN}),
    headers={"Content-Type": "application/json"},
)

# Проверяем, что запрос прошел успешно
if response.status_code == 200:
    # Парсим ответ и извлекаем IAM-токен
    iam_token = response.json()["iamToken"]
    print("IAM-token успешно получен:")
    print(iam_token)
else:
    print(f"Не удалось получить IAM-token. Код ответа: {response.status_code}")
    print("Ответ сервера:")
    print(response.text)


IAM-token успешно получен:
t1.9euelZqNl4rKyIyWj8uVjIyYi8uPme3rnpWajsuPjomPnpmWlcyLm4rKkZLl8_dQOQ1Y-e8eZUhw_N3z9xBoClj57x5lSHD8zef1656VmpfKjZCJm5ONmI2MiZXGyI6J7_zF656VmpfKjZCJm5ONmI2MiZXGyI6J.llqLEfGyAP56XQbvPIjbFYOxJAq3UW01P03pMj8UcNH0ZwwRPS0l_RLDX5HglQm6FKU9agHDyTTq0dfnCPNRDw


In [101]:
import pandas as pd
import requests

IAM_TOKEN = 'y0_AgAAAAAFvktuAATuwQAAAADsR6xABGN80xXHTc682VqotLAtG1s7dvY'
folder_id = '<идентификатор_каталога>'

def translate_texts(texts, target_language):
    body = {
        "targetLanguageCode": target_language,
        "texts": texts,
        "folderId": folder_id,
    }
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {0}".format(IAM_TOKEN)
    }
    
    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', json=body, headers=headers)
    
    if response.status_code == 200:
        res_json = response.json()
        translated_texts = [text['text'] for text in res_json['translations']]
        return translated_texts
    else:
        print(f"Failed to translate texts. Status code: {response.status_code}")
        print("Response text:", response.text)
        return texts

# Загрузите ваш датафрейм
# data = pd.read_csv('path/to/your/data.csv')

# Переведите тексты сначала на английский, затем обратно на русский
data['translated_text_en'] = translate_texts(data['text'].tolist(), 'en')
data['synthesized_text'] = translate_texts(data['translated_text_en'].tolist(), 'ru')

# Удалим промежуточный столбец с английскими переводами
data.drop(columns=['translated_text_en'], inplace=True)

# Теперь у вас есть датафрейм с синтезированными текстами, которые были переведены на английский и обратно на русский


Failed to translate texts. Status code: 429
Response text: {
 "code": 8,
 "message": "grpc: received message larger than max (21546999 vs. 4194304)",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "90f78806-6d6b-4414-8a3d-57ad75e84596"
  }
 ]
}

Failed to translate texts. Status code: 429
Response text: {
 "code": 8,
 "message": "grpc: received message larger than max (21546999 vs. 4194304)",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "c4ff9671-61cb-4eb9-ac61-888ea7a95a17"
  }
 ]
}



In [104]:
import pandas as pd
import requests

# Замените на свои реальные данные
IAM_TOKEN = iam_token
folder_id = '<идентификатор_каталога>'

def translate_texts(texts, target_language):
    body = {
        "targetLanguageCode": target_language,
        "texts": texts,
        "folderId": folder_id,
    }
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {0}".format(IAM_TOKEN)
    }
    
    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', json=body, headers=headers)
    
    if response.status_code == 200:
        res_json = response.json()
        translated_texts = [text['text'] for text in res_json['translations']]
        return translated_texts
    else:
        print(f"Failed to translate texts. Status code: {response.status_code}")
        print("Response text:", response.text)
        return texts

# Загрузите ваш датафрейм
# data = pd.read_csv('path/to/your/data.csv')

# Получите первую строку текста для перевода
first_text = [data.loc[0, 'text']]

# Переведите текст сначала на английский, затем обратно на русский
translated_text_to_en = translate_texts(first_text, 'en')
synthesized_text = translate_texts(translated_text_to_en, 'ru')

# Печать оригинального и синтезированного текстов
print("Original text:", first_text[0])
print("Synthesized text:", synthesized_text[0])


Failed to translate texts. Status code: 403
Response text: {
 "code": 7,
 "message": "Permission denied",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "7dea240b-7e54-498f-a606-f6b7e9607f8c"
  }
 ]
}

Failed to translate texts. Status code: 403
Response text: {
 "code": 7,
 "message": "Permission denied",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "35046249-8388-447f-9928-03055ef959b4"
  }
 ]
}

Original text: Повышение кредитного рейтинга  Акционерного общества «Уральская сталь»  (далее — «Уральская сталь», Компания) вызвано улучшением качественной оценки ликвидности в связи с рефинансированием краткосрочного банковского кредита посредством выпуска облигационного займа с погашением в 2025 году. Также пересмотр стратегических планов по реализации ряда инвестиционных проектов способствовал улучшению показателя «капитальные затраты к выручке». Улучшение ценовой конъюнктуры на мировом рынке чугу

In [105]:
pip install nlpaug

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [106]:
import pandas as pd
import nlpaug.augmenter.word as naw

# Загрузите ваш датафрейм


# Создайте аугментер
aug = naw.SynonymAug(aug_src='wordnet')

def augment_text(text):
    return aug.augment(text)

# Примените аугментацию к каждой строке в столбце 'text'
data['synthesized_text'] = data['text'].apply(augment_text)

# Сохраните исходные данные и синтезированные данные в новом датафрейме
synthesized_data = data[['synthesized_text', 'category', 'rating']]
synthesized_data.columns = ['text', 'category', 'rating']

# Объедините исходные и синтезированные данные
final_data = pd.concat([data[['text', 'category', 'rating']], synthesized_data])

# Сохраните финальный датафрейм
final_data.to_csv('path/to/save/final_data.csv', index=False)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


OSError: Cannot save file into a non-existent directory: 'path\to\save'

In [49]:
category_mapping = {
    'C': 0, 
    'B': 1, 
    'BB': 2, 
    'BBB': 3, 
    'A': 4, 
    'AA': 5, 
    'AAA': 6
}

data['category_num'] = data['category'].replace(category_mapping)

rating_mapping = {
    'C': 0, 
    'B-': 1, 
    'B': 2, 
    'B+': 3, 
    'BB-': 4, 
    'BB': 5, 
    'BB+': 6, 
    'BBB-': 7, 
    'BBB': 8, 
    'BBB+': 9, 
    'A-': 10, 
    'A': 11, 
    'A+': 12, 
    'AA-': 13, 
    'AA': 14, 
    'AA+': 15, 
    'AAA': 16
}

data['rating_num'] = data['rating'].replace(rating_mapping)


In [50]:
data['АКРА_count'] = data['text'].str.count('АКРА')
data['Эксперт_РА_count'] = data['text'].str.count('Эксперт РА')
data['НКР_count'] = data['text'].str.count('НКР')
data['НРА_count'] = data['text'].str.count('НРА')

# Функция для определения агентства с наибольшим числом упоминаний
def most_frequent_agency(row):
    max_count = row.max()
    
    if max_count == 0 or (row == max_count).sum() > 1:
        return 'АКРА'
    else:
        return row.idxmax().replace('_count', '')

# Создание нового столбца с помощью функции
count_columns = ['АКРА_count', 'Эксперт_РА_count', 'НКР_count', 'НРА_count']
data['Agency'] = data[count_columns].apply(most_frequent_agency, axis=1)

# Отображаем первые несколько строк датафрейма для проверки
print(data.head())



   Id                                               text category rating  \
0   1  Повышение кредитного рейтинга  Акционерного об...        A      A   
1   2  «Эксперт РА» подтвердил кредитный рейтинг комп...       BB     BB   
2   3  НКР повысило кредитный рейтинг ООО "ОТЭКО-Порт...        A      A   
3   4  «Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...      AAA    AAA   
4   5  29 марта 2023 г. Ведущий рейтинговый аналитик ...      BBB    BBB   

   category_num  rating_num  АКРА_count  Эксперт_РА_count  НКР_count  \
0             4          11          13                 0          0   
1             2           5           0                15          0   
2             4          11           0                 0         23   
3             6          16           0                16          0   
4             3           8           0                 0          0   

   НРА_count      Agency  
0          0        АКРА  
1          0  Эксперт_РА  
2          0         НКР  
3 

In [51]:
data['Agency'].unique()

array(['АКРА', 'Эксперт_РА', 'НКР', 'НРА'], dtype=object)

In [52]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Zа-яА-Я ]+", "", text).strip()
    text = text.lower()
    return text

data['text'] = data['text'].apply(cleaning)

In [53]:
# from pymystem3 import Mystem
# from nltk.corpus import stopwords
# import nltk
# import pandas as pd
# from tqdm import tqdm

# nltk.download('stopwords')

# m = Mystem()

# # Загрузка русских стоп-слов
# russian_stopwords = stopwords.words("russian")



# def lemmatize_text(corpus):
#     corpus_new = []
#     for sentence in tqdm(corpus):
#         lemmatized_sentence = m.lemmatize(sentence)
#         cleaned_sentence = ' '.join([word for word in lemmatized_sentence if word.strip() and word not in russian_stopwords])
#         corpus_new.append(cleaned_sentence)
#     return corpus_new



# data['text_lemmatized'] = lemmatize_text(data['text'])

# print(data.head())


In [58]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd
import joblib

# Загрузите данные

nltk_stopwords = list(nltk_stopwords.words('russian'))
extra_stopwords = [
    # (Ваш список стоп-слов)
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col, agency):
    # Фильтрация данных по конкретному агентству
    agency_data = data[data['Agency'] == agency]
    
    # Извлекаем признаки и целевую переменную
    X = agency_data['text']
    y = agency_data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    except ValueError:
        # В случае ошибки отключаем стратификацию
        print(f"Stratification failed for {target_col}. Proceeding without stratification.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    ])

    param_grid = [
        {
            'model': [LogisticRegression(random_state=42, solver='liblinear')],
            'model__C': list(range(1, 15, 3)),
            'model__penalty': ['l1', 'l2']
        },
        {
            'model': [SVC(random_state=42)],
            'model__C': [0.1, 1, 10],
            'model__kernel': ['linear', 'rbf']
        },
    ]

    # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Сохраните модель
    joblib.dump(best_grid, f"{agency}_{target_col}_model.joblib")

    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    test_predictions = best_grid.predict(tfidf_test)
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Создание словаря для хранения обученных моделей
models = {}

# Обучите модели для каждого агентства
 
for agency in data['Agency'].unique():
    print(f"Training model for agency: {agency}")
    models[agency] = train_model('category', agency)

test_scores = [scores[1] for scores in models.values()]

# Вычисляем среднее значение
average_test_score = sum(test_scores) / len(test_scores)

print(f"The average test F1 score is: {average_test_score}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for category are: {'model': SVC(C=10, kernel='linear', random_state=42), 'model__C': 10, 'model__kernel': 'linear'}
Best cross-validation score for category is: 0.7785740652466765
Test F1 score for category is: 0.8241440265049673
Final TEST F1 score for 'category': 0.8241440265049673


In [67]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd
import joblib

nltk_stopwords = list(nltk_stopwords.words('russian'))
extra_stopwords = [
        'далее', 'также', 'что', 'в', 'с', 'и', 'на', 'по', 'а', 'за', '—', 'как', 
    'у', 'до', 'средней', 'очень', 'один', 'это', 'средняя', '«', '»', '(', ')', 
    '№', '—', 'году', 'одной', 'посредством', 'ранее', 'большого', 'которая', 
    'который', 'этом', 'является', 'один', 'другой', 'доли', 'доля', 'их', 
    'которого', 'его', 'средний', 'средние', 'уровень', 'производителей', 
    'высокие', 'низкой', 'средней', 'продукции', 'компании', 'компания', 
    'продукция', 'оценка', 'оценки', 'оценку', 'показатели', 'показатель', 
    'факторы', 'фактор', 'уровня', 'профиля', 'бизнес', 'бизнеса', 'уровень', 
    'уровня', 'рейтинг', 'рейтинга', 'рейтингу', 'производства', 'производство'
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    ])

    param_grid = [
        {
            'model': [LogisticRegression(random_state=42, solver='liblinear')],
            'model__C': list(range(1, 15, 3)),
            'model__penalty': ['l1', 'l2']
        },
        {
            'model': [SVC(random_state=42)],
            'model__C': [0.1, 1, 10],
            'model__kernel': ['linear', 'rbf']
        },
    ]

    # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    test_predictions = best_grid.predict(tfidf_test)
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    joblib.dump(best_grid.best_estimator_, f'best_model_{target_col}.joblib')
    return grid.best_score_, test_score

# Получите лучшие оценки для 'category'
category_cv_f1, category_test_f1 = train_model('category')

print(f"Final CV F1 score for 'category': {category_cv_f1}")
print(f"Final TEST F1 score for 'category': {category_test_f1}")

best_model = joblib.load('best_model_category.joblib')

# Примените TF-IDF векторизатор к вашим данным
tfidf_data = count_tf_idf.transform(data['text'])

# Предсказание категорий
data['category_pred'] = best_model.predict(tfidf_data)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for category are: {'model': SVC(C=10, kernel='linear', random_state=42), 'model__C': 10, 'model__kernel': 'linear'}
Best cross-validation score for category is: 0.7785740652466765
Test F1 score for category is: 0.8241440265049673
Final CV F1 score for 'category': 0.7785740652466765
Final TEST F1 score for 'category': 0.8241440265049673


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords

# Загружаем стоп-слова
nltk.download('stopwords')
nltk_stopwords = list(stopwords.words('russian'))
extra_stopwords = [
        'далее', 'также', 'что', 'в', 'с', 'и', 'на', 'по', 'а', 'за', '—', 'как', 
    'у', 'до', 'средней', 'очень', 'один', 'это', 'средняя', '«', '»', '(', ')', 
    '№', '—', 'году', 'одной', 'посредством', 'ранее', 'большого', 'которая', 
    'который', 'этом', 'является', 'один', 'другой', 'доли', 'доля', 'их', 
    'которого', 'его', 'средний', 'средние', 'уровень', 'производителей', 
    'высокие', 'низкой', 'средней', 'продукции', 'компании', 'компания', 
    'продукция', 'оценка', 'оценки', 'оценку', 'показатели', 'показатель', 
    'факторы', 'фактор', 'уровня', 'профиля', 'бизнес', 'бизнеса', 'уровень', 
    'уровня', 'рейтинг', 'рейтинга', 'рейтингу', 'производства', 'производство'
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

X_text_tfidf = count_tf_idf.fit_transform(data['text'])

# Применяем OneHotEncoder
ohe = OneHotEncoder()
X_category_ohe = ohe.fit_transform(data[['category_pred']])

# Объединяем преобработанные признаки
from scipy.sparse import hstack
X = hstack([X_text_tfidf, X_category_ohe])

# Разделяем данные на тренировочные и тестовые наборы
y = data['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

model = SVC(C=10, kernel='linear', random_state=42)
model.fit(X_train, y_train)

# Оцениваем модель на тестовом наборе
y_pred = model.predict(X_test)
test_f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'Test F1 score: {test_f1_score}')

joblib.dump(model, 'model_rating.joblib')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Test F1 score: 0.7427151024753764


['model_rating.joblib']

In [94]:
final_test_score = (test_f1_score * 0.65) + (category_test_f1 * 0.35) 
final_test_score

0.7712152258857332

In [33]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd

# Загрузите данные
data = pd.read_csv('data.csv')

nltk_stopwords = list(nltk_stopwords.words('russian'))
extra_stopwords = [
    'далее', 'также', 'что', 'в', 'с', 'и', 'на', 'по', 'а', 'за', '—', 'как', 
    'у', 'до', 'средней', 'очень', 'один', 'это', 'средняя', '«', '»', '(', ')', 
    '№', '—', 'году', 'одной', 'посредством', 'ранее', 'большого', 'которая', 
    'который', 'этом', 'является', 'один', 'другой', 'доли', 'доля', 'их', 
    'которого', 'его', 'средний', 'средние', 'уровень', 'производителей', 
    'высокие', 'низкой', 'средней', 'продукции', 'компании', 'компания', 
    'продукция', 'оценка', 'оценки', 'оценку', 'показатели', 'показатель', 
    'факторы', 'фактор', 'уровня', 'профиля', 'бизнес', 'бизнеса', 'уровень', 
    'уровня', 'рейтинг', 'рейтинга', 'рейтингу', 'производства', 'производство'
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    ])

    param_grid = [
    {
        'model': [LogisticRegression(random_state=42, solver='liblinear')],
        'model__C': list(range(1, 15, 3)),
        'model__penalty': ['l1', 'l2']
    },

    {
        'model': [SVC(random_state=42)],
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf']
    },

    ]

        # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    tfidf_test = count_tf_idf.transform(X_test)
    test_predictions = best_grid.predict(tfidf_test)
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Получите лучшие оценки для каждой целевой переменной
rating_cv_f1, rating_test_f1 = train_model('rating')
category_cv_f1, category_test_f1 = train_model('category')

# Рассчитайте взвешенный итоговый результат
final_cv_score = (rating_cv_f1 * 0.65) + (category_cv_f1 * 0.35)
final_test_score = (rating_test_f1 * 0.65) + (category_test_f1 * 0.35)

print(f"Final CV F1 score for 'rating': {rating_cv_f1}")
print(f"Final CV F1 score for 'category': {category_cv_f1}")
print(f"Overall final CV score: {final_cv_score}")

print(f"Final TEST F1 score for 'rating': {rating_test_f1}")
print(f"Final TEST F1 score for 'category': {category_test_f1}")
print(f"Overall final TEST score: {final_test_score}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for rating are: {'model': SVC(C=10, kernel='linear', random_state=42), 'model__C': 10, 'model__kernel': 'linear'}
Best cross-validation score for rating is: 0.6106691542561378
Test F1 score for rating is: 0.6764194855512256
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for category are: {'model': SVC(C=10, kernel='linear', random_state=42), 'model__C': 10, 'model__kernel': 'linear'}
Best cross-validation score for category is: 0.7785740652466765
Test F1 score for category is: 0.8241440265049673
Final CV F1 score for 'rating': 0.6106691542561378
Final CV F1 score for 'category': 0.7785740652466765
Overall final CV score: 0.6694358731028263
Final TEST F1 score for 'rating': 0.6764194855512256
Final TEST F1 score for 'category': 0.8241440265049673
Overall final TEST score: 0.7281230748850351


In [9]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error, r2_score, f1_score, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd

# Загрузите данные и мэппинги
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('russian')



count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LinearRegression())
    ])

    param_grid = [
        {
            'model': [LinearRegression()],
        },
        {
            'model': [RandomForestRegressor(random_state=42)],
            'model__n_estimators': [100],
            'model__max_depth': [None]
        },

    ]

    # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring='r2', cv=5, verbose=10, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    test_predictions = best_grid.predict(tfidf_test)

    # Восстановим буквенные рейтинги
    inv_category_mapping = {v: k for k, v in category_mapping.items()}
    inv_rating_mapping = {v: k for k, v in rating_mapping.items()}

    if target_col == 'category_num':
        test_predictions = np.clip(test_predictions, 0, 6)
        test_predictions = [inv_category_mapping[round(pred)] for pred in test_predictions]
        y_test = [inv_category_mapping[val] for val in y_test]
    elif target_col == 'rating_num':
        test_predictions = np.clip(test_predictions, 0, 16)
        test_predictions = [inv_rating_mapping[round(pred)] for pred in test_predictions]
        y_test = [inv_rating_mapping[val] for val in y_test]

    # Вычислим F1-оценку
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Получите лучшие оценки для каждой целевой переменной
rating_cv_r2, rating_test_f1 = train_model('rating_num')
category_cv_r2, category_test_f1 = train_model('category_num')

# Выведите итоговые результаты
print(f"Final CV R2 score for 'rating': {rating_cv_r2}")
print(f"Final CV R2 score for 'category': {category_cv_r2}")

print(f"Final TEST F1 score for 'rating': {rating_test_f1}")
print(f"Final TEST F1 score for 'category': {category_test_f1}")



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters for rating_num are: {'model': LinearRegression()}
Best cross-validation score for rating_num is: 0.8364905740074908
Test F1 score for rating_num is: 0.4992087191172545
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters for category_num are: {'model': LinearRegression()}
Best cross-validation score for category_num is: 0.8082170222794562
Test F1 score for category_num is: 0.7873336567336569
Final CV R2 score for 'rating': 0.8364905740074908
Final CV R2 score for 'category': 0.8082170222794562
Final TEST F1 score for 'rating': 0.4992087191172545
Final TEST F1 score for 'category': 0.7873336567336569


In [5]:
data = pd.read_csv('data.csv')

Рассмотрим несколько стратегий для создания новых признаков:

Использование Финансовых Терминов
Частота финансовых терминов: Создайте признаки, которые представляют собой количество упоминаний ключевых финансовых терминов, таких как "долг", "прибыль", "риск" и т. д., в каждом тексте.

Sentiment Analysis на финансовых терминах: Проведите анализ тональности сосредоточиваясь на предложениях, в которых упоминаются финансовые термины, чтобы оценить, является ли контекст положительным, отрицательным или нейтральным.

Использование Метрических Данных
Числовые метрики: Создайте признаки, представляющие количество числовых упоминаний в тексте, или даже более сложные метрики, такие как среднее или медианное значение упоминаемых чисел.

Упоминание финансовых показателей: Определите, упоминаются ли специфические финансовые показатели (например, EBITDA, P/E ratio) и создайте бинарные признаки на основе их наличия или отсутствия.

Прочие Стратегии
Кластеризация текстов: Попробуйте использовать алгоритмы кластеризации для группировки текстов по схожести тем, и используйте метки кластеров как признаки.

Создание тематических дикционариев: Создайте словари с положительными и отрицательными финансовыми терминами и используйте их для создания признаков на основе числа положительных и отрицательных слов в тексте.

Длина текста и структура предложения: Включите признаки, связанные с длиной текста или структурой предложения (например, средняя длина предложения).

In [7]:
import pandas as pd

# Список признаков и соответствующие им ключевые слова для поиска в тексте
features_keywords = {
    "mlrd": ["млрд"],
    "mln": ["млн"],
    "rating_assessment": ["rating"],
    "regulatory_requirements": ["регуляторный"],
    "disclosure": ["раскрытие"],
    "credit_rating": ["кредитный"],
    "joint_stock_company": ["общество"],
    "bond_issue": ["облигаций"],
    "RUAQ": ["RUAQ"],
    "national_scale": ["шкала"],
    "non_financial_company": ["компания"],
    "agency": ["агентство"],
    "rating_activity": ["деятельность"],
    "financial_instrument": ["финансовый"],
    "AKR_credit_rating": ["AKR"],
    "credit_rating_forecast": ["прогноз"],
    "press_release": ["пресс-релиз"],
    "open_source_information": ["информация"],
    "AKR_database": ["база данных"],
    "reporting": ["отчетность"],
    "additional_services": ["услуги"],
    "conflict_of_interest": ["интересов"],
    "connection_category": ["категория"],
    "level": ["уровень"],
    "senior_unsecured_debt": ["долг"],
    "emission": ["эмиссии"],
    }

for feature, keywords in features_keywords.items():
    data[feature] = data["text_lemmatized"].apply(lambda x: any(keyword in x for keyword in keywords))
print(data.head())


   Unnamed: 0  Id                                               text category  \
0           0   1  повышение кредитного рейтинга  акционерного об...        A   
1           1   2  эксперт ра подтвердил кредитный рейтинг компан...       BB   
2           2   3  нкр повысило кредитный рейтинг ооо отэкопортсе...        A   
3           3   4  эксперт ра присвоил кредитный рейтинг пао фоса...      AAA   
4           4   5  марта  г ведущий рейтинговый аналитик юрова ал...      BBB   

  rating                                    text_lemmatized   mlrd    mln  \
0      A  повышение кредитный рейтинг акционерный общест...   True  False   
1     BB  эксперт ра подтверждать кредитный рейтинг комп...  False   True   
2      A  нкр повышать кредитный рейтинг ооо отэкопортсе...  False   True   
3    AAA  эксперт ра присваивать кредитный рейтинг пао ф...   True   True   
4    BBB  марта г ведущий рейтинговый аналитик юров алла...   True  False   

   rating_assessment  regulatory_requirements  ...